In [ ]:
pip install PyPortfolioOpt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import black_litterman
import pandas as pd
import numpy as np
import statsmodels.api as sm
from pandas.tseries.offsets import *
import datetime
#import scipy
pd.options.mode.chained_assignment = None  # turnoff false postive warnging

Connect google dirve to colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Read in stock price/return data

In [ ]:
#import month-end prices for 12/2012-12/2022 (121 monthly obs for each stock)
prc = pd.read_csv("/content/drive/MyDrive/GEPM_DATA/FinalProject/stock_prices.csv", parse_dates=True, index_col="Date")
#calculate monthly returns
ret = (prc-prc.shift())/prc.shift() #shift(1) produces the same results
ret = ret.dropna() #drop obs with missing return data
ret = ret.reset_index() #reset index and keep date as a datetime variable
ret.columns = map(str.lower, ret.columns)#convert variable names to lower cases
ret

,date,amt,blk,cat,jnj,msft,nke,tjx,tsla,vz,xom
0,2013-01-03,-0.024394,0.002737,0.009641,-0.001307,-0.013644,0.010376,0.032909,-0.016949,-0.004410,-0.001735
1,2013-01-04,-0.000944,0.021961,0.005477,0.011402,-0.018742,0.009842,-0.000514,-0.012931,0.005168,0.004519
2,2013-01-07,-0.001259,-0.001840,0.003073,-0.002033,-0.001819,0.001271,-0.006684,0.000000,0.008814,-0.011592
3,2013-01-08,0.004098,-0.015522,-0.012671,0.000000,-0.005011,-0.010580,-0.017081,-0.017467,-0.024390,0.006301
4,2013-01-09,0.005965,0.002658,0.004654,0.004630,0.005495,0.001283,0.003686,-0.004444,-0.002239,-0.003827
...,...,...,...,...,...,...,...,...,...,...,...
2512,2022-12-23,0.008802,0.000874,0.009813,0.002503,0.002273,-0.003953,0.013823,-0.017551,0.002695,0.026459
2513,2022-12-27,0.000047,-0.000973,0.013656,-0.000284,-0.007391,0.011301,-0.005934,-0.114089,0.021774,0.013908
2514,2022-12-28,-0.003793,-0.004168,-0.015497,-0.004314,-0.010280,-0.022008,-0.004318,0.033089,-0.011050,-0.016461
2515,2022-12-29,0.026844,0.022595,0.003442,0.005074,0.027657,0.020672,0.008291,0.080827,0.011439,0.007624


In [ ]:
#import month-end market cap for 01/2013-12/2017 (60 monthly obs for each stock)
mktcap = pd.read_csv("/content/drive/MyDrive/GEPM_DATA/FinalProject/marketcap2017.csv", parse_dates=True, index_col="date")
mktcap = mktcap.dropna()
mktcap = mktcap.reset_index() #reset index and keep date as a datetime variable
mktcap.columns = map(str.lower, mktcap.columns)#convert variable names to lower cases
mktcap

,date,permno,prc,shrout
0,2013-01-31,10107,27.45000,8376245
1,2013-02-28,10107,27.80000,8376245
2,2013-03-28,10107,28.60500,8349000
3,2013-04-30,10107,33.10000,8351107
4,2013-05-31,10107,34.90000,8351107
...,...,...,...,...
595,2017-08-31,93436,355.89999,166887
596,2017-09-29,93436,341.10001,168017
597,2017-10-31,93436,331.53000,168067
598,2017-11-30,93436,308.85001,168067


In [ ]:
# Calculate market cap by multiplying per-share price by total shares outstanding
mktcap['market_cap'] = mktcap['prc'] * mktcap['shrout']
temp = mktcap.groupby("permno")["date", "market_cap"].agg(list)
permno_name_mapping = {10107:"amt", 11850:"blk",18542: "cat",22111: "jnj",40539: "msft",57665: "nke",65875: "tjx",86111: "tsla",87267: "vz",93436: "xom"}
mc_dict = {"date":[]}
for perm_no in temp.index:
  mc_dict[permno_name_mapping[perm_no]] = temp.loc[perm_no]["market_cap"]
mc_dict["date"] = temp.loc[perm_no]["date"]
mktcap_df = pd.DataFrame(mc_dict)
mktcap_df.head(5)

<ipython-input-31-6a8906295b11>:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  temp = mktcap.groupby("permno")["date", "market_cap"].agg(list)


,date,amt,blk,cat,jnj,msft,nke,tjx,tsla,vz,xom
0,2013-01-31,2.299279e+08,4.102041e+08,64340566.26,2.048517e+08,32947650.54,38695854.35,1.246810e+08,30106130.95,4.045492e+07,4295570.18
1,2013-02-28,2.328596e+08,4.012243e+08,60506783.76,2.127517e+08,32794507.41,38989384.42,1.329955e+08,30659992.80,4.104904e+07,3988661.94
2,2013-03-28,2.388231e+08,4.037333e+08,56969524.56,2.280423e+08,34092577.75,42246852.27,1.404842e+08,30391322.76,4.365958e+07,4363450.29
3,2013-04-30,2.764216e+08,3.956495e+08,55687459.00,2.383914e+08,35148831.62,45515785.20,1.542365e+08,33227535.87,4.503130e+07,6238706.47
4,2013-05-31,2.914536e+08,4.022636e+08,56430660.00,2.364521e+08,36427711.53,44127410.62,1.387013e+08,30784630.24,4.710216e+07,11296461.28


In [ ]:
#import excess market return and risk-free rate (Ken French online data lib)
mktrf = pd.read_csv("/content/drive/MyDrive/GEPM_DATA/FinalProject/mktrf.csv", parse_dates=True, index_col="date")
mktrf = mktrf.reset_index() #reset index, and keep date as a datetime variable
mktrf.columns = map(str.lower, mktrf.columns)#convert variable names to lower cases
mktrf['mkt_rf'] = mktrf['mkt_rf']/100
mktrf['rf'] = mktrf['rf']/100
mktrf

,date,mkt_rf,rf
0,1926-07-01,0.0010,0.00009
1,1926-07-02,0.0045,0.00009
2,1926-07-06,0.0017,0.00009
3,1926-07-07,0.0009,0.00009
4,1926-07-08,0.0021,0.00009
...,...,...,...
25433,2023-02-22,-0.0004,0.00018
25434,2023-02-23,0.0046,0.00018
25435,2023-02-24,-0.0109,0.00018
25436,2023-02-27,0.0031,0.00018


In [ ]:
#calculate each stock's market share
#for each month, sum up market caps across all stocks
totcap = mktcap_df.iloc[:, 1:].sum(axis=1)
totcap

0     1.180505e+09
1     1.187820e+09
2     1.222806e+09
3     1.285549e+09
4     1.295040e+09
5     1.295779e+09
6     1.315411e+09
7     1.268580e+09
8     1.278858e+09
9     1.348821e+09
10    1.387674e+09
11    1.413520e+09
12    1.357720e+09
13    1.466004e+09
14    1.505089e+09
15    1.522115e+09
16    1.531772e+09
17    1.552751e+09
18    1.542049e+09
19    1.589147e+09
20    1.578435e+09
21    1.604643e+09
22    1.596697e+09
23    1.554673e+09
24    1.449060e+09
25    1.511757e+09
26    1.457366e+09
27    1.533573e+09
28    1.510479e+09
29    1.463052e+09
30    1.475471e+09
31    1.399184e+09
32    1.388202e+09
33    1.544869e+09
34    1.550635e+09
35    1.536710e+09
36    1.537207e+09
37    1.519995e+09
38    1.610318e+09
39    1.585418e+09
40    1.606667e+09
41    1.653933e+09
42    1.702454e+09
43    1.665870e+09
44    1.654687e+09
45    1.623971e+09
46    1.642682e+09
47    1.698011e+09
48    1.676058e+09
49    1.701429e+09
50    1.724218e+09
51    1.748628e+09
52    1.7744

In [ ]:
#copy to keep the columns
vw = mktcap_df.copy()
#cal. weight and replace mkt cap with weight
vw.iloc[:, 1:] = mktcap_df.iloc[:, 1:].div(totcap, axis=0)
vw

,date,amt,blk,cat,jnj,msft,nke,tjx,tsla,vz,xom
0,2013-01-31,0.194771,0.347482,0.054503,0.173529,0.027910,0.032779,0.105617,0.025503,0.034269,0.003639
1,2013-02-28,0.196040,0.337782,0.050939,0.179111,0.027609,0.032824,0.111966,0.025812,0.034558,0.003358
2,2013-03-28,0.195307,0.330169,0.046589,0.186491,0.027881,0.034549,0.114887,0.024854,0.035704,0.003568
3,2013-04-30,0.215022,0.307767,0.043318,0.185439,0.027342,0.035406,0.119977,0.025847,0.035029,0.004853
4,2013-05-31,0.225054,0.310619,0.043574,0.182583,0.028129,0.034074,0.107102,0.023771,0.036371,0.008723
5,2013-06-28,0.222021,0.310030,0.041870,0.186120,0.027807,0.035170,0.111148,0.022332,0.033441,0.010061
6,2013-07-31,0.201630,0.316896,0.041455,0.199657,0.028476,0.034076,0.107656,0.021257,0.036501,0.012398
7,2013-08-30,0.219317,0.302422,0.042137,0.191955,0.029724,0.035278,0.106892,0.021637,0.034459,0.016180
8,2013-09-30,0.217190,0.296136,0.042234,0.191029,0.031538,0.040465,0.104446,0.022896,0.035534,0.018533
9,2013-10-31,0.219124,0.292458,0.040024,0.193489,0.032235,0.039969,0.107165,0.023216,0.037782,0.014537


Partition sample into estimation and test samples

In [ ]:
#estimation sample (first five years)
#df_est = ret[(ret['Date']>=datetime.datetime(2013, 1, 31)) & (ret['Date']<=datetime.datetime(2017, 12, 31))].copy()
df_est = ret.iloc[:60, :]

In [ ]:
#test/evaluation sample: (last five years)
#df_eva = ret[(ret['Date']>=datetime.datetime(2018, 1, 31)) & (ret['Date']<=datetime.datetime(2022, 12, 31))].copy()
df_eva = ret.iloc[60:, :]

Stock weights based on alternative approaches

1. Value-weighted portfolio

In [ ]:
#w1 = vw[(vw['date']>=datetime.datetime(2017, 12, 1)) & (vw['date']<=datetime.datetime(2017, 12, 31))]
w1 = vw[vw['date']==datetime.datetime(2017, 12, 29)] #last trading date in Dec 2017 or the portfolio formation date
w1 = w1.drop('date', axis=1)
w1

,amt,blk,cat,jnj,msft,nke,tjx,tsla,vz,xom
59,0.325468,0.175005,0.046295,0.18536,0.023874,0.04053,0.106628,0.030214,0.040673,0.025953


2. Equal-weighted portfolio

In [ ]:
#w2: equal-weighted as of end of 12/2017
w2 = w1.copy() #keep the same template
shape = w2.shape
N = shape[1] #count number of stocks
w2[:] = 1/N
w2

,amt,blk,cat,jnj,msft,nke,tjx,tsla,vz,xom
59,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1


3. Glboal minimum variance portfolio

In [ ]:
#w3: weights of global minimum variance
#step 1: calculate expected returns and sample covariance
tmp1 = df_est.iloc[:, 1:] #keep only return columns in the estimation sample
mu = expected_returns.mean_historical_return(tmp1, returns_data=True, compounding=False, frequency=12) #annualized
print(mu)
S = risk_models.sample_cov(tmp1, returns_data=True, frequency=12) #annualized
print(S)
###
#step 2: Optimize for maximal Sharpe ratio
ef = EfficientFrontier(mu, S, weight_bounds=(0, 1)) #default contraints
raw_weights = ef.min_volatility()
###
#step 3: convert ordered dictionary into dataframe
w3=pd.DataFrame(raw_weights, index=[0]) #use scalar value and pass an index
w3 #1XN vector

amt    -0.002665
blk     0.037031
cat    -0.016896
jnj     0.030897
msft    0.009111
nke     0.025904
tjx     0.017524
tsla    0.048919
vz      0.024009
xom     0.006143
dtype: float64
           amt       blk       cat       jnj      msft       nke       tjx  \
amt   0.001314  0.000094  0.000250  0.000209  0.000378  0.000278  0.000219   
blk   0.000094  0.002002  0.000590  0.000311  0.000378  0.000356  0.000434   
cat   0.000250  0.000590  0.001332  0.000139  0.000263  0.000106  0.000501   
jnj   0.000209  0.000311  0.000139  0.000306  0.000147  0.000260  0.000138   
msft  0.000378  0.000378  0.000263  0.000147  0.000832  0.000086  0.000181   
nke   0.000278  0.000356  0.000106  0.000260  0.000086  0.003661  0.000719   
tjx   0.000219  0.000434  0.000501  0.000138  0.000181  0.000719  0.000877   
tsla  0.000616  0.000599  0.000378  0.000316  0.000807 -0.000369  0.000506   
vz    0.000048  0.000223  0.000222  0.000078  0.000060 -0.000222  0.000046   
xom   0.000342  0.000521  0.000331 

,amt,blk,cat,jnj,msft,nke,tjx,tsla,vz,xom
0,0.018423,0.0,0.00595,0.544114,0.120024,0.005227,0.120206,0.0,0.186057,0.0


4. Maximal Sharpe ratio portfolio

In [ ]:
#w4: weights of maximal sharpe ratio
#step 1 (skipped): calculate expected returns and sample covariance
###
#step 2: Optimize for maximal Sharpe ratio
ef = EfficientFrontier(mu, S, weight_bounds=(0, 1)) #default contraints
raw_weights = ef.max_sharpe()
###
#step 3: convert ordered dictionary into dataframe
w4=pd.DataFrame(raw_weights, index=[0]) #use scalar value and pass an index
w4 #1XN vector

,amt,blk,cat,jnj,msft,nke,tjx,tsla,vz,xom
0,0.0,0.088461,0.0,0.841936,0.0,0.0,0.0,0.041089,0.028515,0.0


5. Black-Litterman expected rets, applied to maximal Sharpe Ratio portfolio

In [ ]:
#using market data up to portfolio formation (avoid look-ahead bias)
stats_mktrf = mktrf.loc[:60, 'mkt_rf'].describe()
delta = stats_mktrf.loc['mean']*12/(stats_mktrf.loc['std']*12**0.5)
delta

0.6650170033552751

In [ ]:
#implied equilibrium risk premium
mcaps = mktcap_df[mktcap_df['date']==datetime.datetime(2017, 12, 29)].copy()
mcaps = mcaps.drop('date', axis=1) #drop Date variable
mcaps = mcaps.squeeze() #convert df to series
prior = black_litterman.market_implied_prior_returns(mcaps, 2, S)
print(prior)

amt     0.021135
blk     0.021153
cat     0.020730
jnj     0.020468
msft    0.020608
nke     0.020841
tjx     0.020694
tsla    0.021624
vz      0.020229
xom     0.020704
dtype: float64


In [ ]:
#investor views
#where did we get this from???
viewdict = {"amt": 0.14, "jnj": 0.10, "tjx": 0.25, "tsla": 0.08}

In [ ]:
#posterior returns
from pypfopt.black_litterman import BlackLittermanModel
bl = BlackLittermanModel(S, prior, absolute_views=viewdict)
posterior = bl.bl_returns()
posterior

amt     0.102957
blk     0.104058
cat     0.100385
jnj     0.071181
msft    0.065131
nke     0.140535
tjx     0.144407
tsla    0.104868
vz      0.034478
xom     0.084075
dtype: float64

In [ ]:
#finding weights for maximal sharpe ratio portofolio
ef = EfficientFrontier(posterior, S, weight_bounds=(0, 1)) #default contraints
raw_weights = ef.max_sharpe()
#convert ordered dictionary into dataframe
w5=pd.DataFrame(raw_weights, index=[0]) #use scalar value and pass an index
w5

,amt,blk,cat,jnj,msft,nke,tjx,tsla,vz,xom
0,0.116399,0.000318,0.001246,0.383649,0.0,0.003157,0.49299,0.0,0.002241,0.0


6. Covariance shrinkage method, applied to global minimum variance portfolio

In [ ]:
#shrink covariance
S_sk = risk_models.CovarianceShrinkage(df_est.iloc[:, 1:], returns_data=True, frequency=12).ledoit_wolf()
S_sk

,amt,blk,cat,jnj,msft,nke,tjx,tsla,vz,xom
amt,0.001624,0.000062,0.000165,0.000138,0.000250,0.000183,0.000145,0.000407,0.000032,0.000226
blk,0.000062,0.002078,0.000390,0.000206,0.000250,0.000235,0.000287,0.000395,0.000148,0.000344
cat,0.000165,0.000390,0.001635,0.000092,0.000173,0.000070,0.000331,0.000250,0.000147,0.000219
jnj,0.000138,0.000206,0.000092,0.000958,0.000097,0.000171,0.000091,0.000209,0.000052,0.000174
msft,0.000250,0.000250,0.000173,0.000097,0.001305,0.000057,0.000120,0.000533,0.000040,0.000154
nke,0.000183,0.000235,0.000070,0.000171,0.000057,0.003174,0.000475,-0.000244,-0.000147,0.000203
tjx,0.000145,0.000287,0.000331,0.000091,0.000120,0.000475,0.001334,0.000335,0.000030,0.000168
tsla,0.000407,0.000395,0.000250,0.000209,0.000533,-0.000244,0.000335,0.008463,-0.000008,0.000517
vz,0.000032,0.000148,0.000147,0.000052,0.000040,-0.000147,0.000030,-0.000008,0.001342,0.000043
xom,0.000226,0.000344,0.000219,0.000174,0.000154,0.000203,0.000168,0.000517,0.000043,0.001105


In [ ]:
#Optimize for global minimum variance
ef = EfficientFrontier(mu, S_sk, weight_bounds=(0, 1)) #default contraints
raw_weights = ef.min_volatility()
#convert ordered dictionary into dataframe
w6=pd.DataFrame(raw_weights, index=[0]) #use scalar value and pass an index
w6

,amt,blk,cat,jnj,msft,nke,tjx,tsla,vz,xom
0,0.089273,0.027134,0.071885,0.200702,0.134649,0.049897,0.112857,0.001343,0.183439,0.12882


7. Combine BL and SC, applied to maximal Sharpe ratio portfolio

In [ ]:
ef = EfficientFrontier(posterior, S_sk, weight_bounds=(0, 1)) #default contraints
raw_weights = ef.max_sharpe()
w7=pd.DataFrame(raw_weights, index=[0]) #use scalar value and pass an index
w7

,amt,blk,cat,jnj,msft,nke,tjx,tsla,vz,xom
0,0.144558,0.069367,0.095827,0.122552,0.043653,0.097522,0.299827,0.011228,0.021633,0.093833


Performance evaluation

Define performance evaluation function

In [ ]:
def perf_eval(weight, outfile):
  #calculate ex-post monthly portfolio return for value-weighted portfolio
  r_p = pd.DataFrame() #create a dataframe to save monthly portfolio returns
  r_p['r_p'] = df_eva.iloc[:,1:].dot(weight.T) #matrix multiplication (df_eva is a TXN matrix, wt is a 1xN vector)
  r_p['date'] = df_eva['date'] #get the date variable from the test sample
  #r_p
  ##
  #merge portfolio, market, and risk-free return data
  tmp = r_p.merge(mktrf, how = 'inner') #add excess market return and risk-free rate
  tmp['r_p'] = (tmp['r_p'] - tmp['rf'])
  tmp = tmp.rename(columns = {'mkt_rf': 'r_m'}) #rename excess market return to r_m
  tmp [['r_m', 'r_p']] = tmp[['r_m', 'r_p']]*12 #annualize return
  data = tmp[['date', 'r_m', 'r_p']] #keep the variables of interest
  #data
  ##
  #summary stats (e.g., mean, std, min, max); 1%, 2.5%, and 5% Value-at-Risk
  stat = data.describe(percentiles = [0.01, 0.025, 0.05])
  #print(stat)
  ##
  #Sharpe ratio
  sharpe = stat.loc['mean']/stat.loc['std']
  sharpe = sharpe.rename('sharpe')
  #print(sharpe)
  ##
  #estimate market model
  data['constant'] = 1
  Y = data['r_p']
  X = data[['constant', 'r_m']]
  result = sm.OLS(Y,X).fit()
  #print(result.summary())
  #keep the results of interest
  from statsmodels.iolib.summary2 import summary_col
  regout = summary_col(result, regressor_order=result.params.index.tolist())
  regout = regout.tables[0] #convert into a table
  ##
  #Treynor ratio
  params = result.params #obtain slope coef from regression
  tr = stat.loc[['mean'],['r_p']]/params.loc['r_m']
  tr = tr.rename(columns={'r_p':'tr'})
  tr = tr.reset_index(drop=True)
  #print(tr)
  ##
  #information ratio
  sig_e = (result.resid).std() #cal. std dev of reg residuals
  ir = stat.loc[['mean'],['r_p']]/sig_e
  ir = ir.rename(columns={'r_p':'ir'})
  ir = ir.reset_index(drop=True)
  #print(ir)
  ##
  #market timing ability
  #returns for up market
  data['up']= data['r_m']
  data.loc[data['r_m'] < 0, 'up'] = 0
  #returns for down market
  data['dn']= data['r_m']
  data.loc[data['r_m'] >= 0, 'dn'] = 0
  #regression analysis
  Y = data['r_p']
  X = data[['constant', 'up', 'dn']]
  result_mt = sm.OLS(Y,X).fit()
  #print(result_mt.summary())
  ###
  #keep the results of interest
  regout_mt = summary_col(result_mt, regressor_order=result_mt.params.index.tolist())
  regout_mt = regout_mt.tables[0]
  ##
  writer = pd.ExcelWriter('//content//drive//MyDrive//GEPM_DATA//FinalProject//' + \
          outfile + '.xlsx', engine='openpyxl')
  stat.to_excel(writer, sheet_name='stat')
  sharpe.to_excel(writer, sheet_name='sharpe')
  regout.to_excel(writer, sheet_name='regout')
  tr.to_excel(writer, sheet_name='tr')
  ir.to_excel(writer, sheet_name='ir')
  regout_mt.to_excel(writer, sheet_name='regout_mt')
  writer.save()

In [ ]:
perf_eval(w1, 'perf_eva_vw') #value-weighted portfolio
perf_eval(w2, 'perf_eva_ew') #equal-weighted portfolio
perf_eval(w3, 'perf_eva_gmv') #global minimum variance portfolio
perf_eval(w4, 'perf_eva_maxsrp') #maximal sharpe ratio portfolio
perf_eval(w5, 'perf_eva_bl') #black-litterman, maximal sharpe ratio
perf_eval(w6, 'perf_eva_sc') #shrinked covariance, gmv
perf_eval(w7, 'perf_eva_bl+sc') #shrinked covariance, gmv

<ipython-input-56-138dec2667e0>:75: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
<ipython-input-56-138dec2667e0>:75: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
<ipython-input-56-138dec2667e0>:75: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
<ipython-input-56-138dec2667e0>:75: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
<ipython-input-56-138dec2667e0>:75: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
<ipython-input-56-138dec2667e0>:75: FutureWarning: save is not part of the public API, usage can give unexpected re